In [2]:
# !pip3 install -U openai-whisper -i https://mirrors.aliyun.com/pypi/simple/
# !pip3 install jiwer -i https://mirrors.aliyun.com/pypi/simple/

In [18]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
from tqdm import tqdm
import torch
import librosa
import numpy as np
import glob
import os

import sys
sys.path.append("/home/shoinoue/Git/cuhksz-phd/sho_util/pyfiles/")
from sound import play_audio

from whisper.normalizers.english import EnglishTextNormalizer
normalizer = EnglishTextNormalizer()
import jiwer
import pandas as pd
import scipy.stats as st
import math
from IPython.display import clear_output

from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from datasets import load_dataset
import soundfile as sf
from scipy.io.wavfile import write as write_wav

sr = 16000

class FacebookASR():
    def __init__(self, fs=16000, model_path="/mntcephfs/data/audiow/shoinoue/Model/hf_hub/models--facebook--s2t-large-librispeech-asr/snapshots/a4b4750ad1425acda0dbd1daa9188d5fd7872491"):
        self.model = Speech2TextForConditionalGeneration.from_pretrained(model_path).to("cuda")
        self.processor = Speech2TextProcessor.from_pretrained(model_path)
        self.fs = fs
        
    def get_sample(self, file):
        x, _ = librosa.load(file)
        write_wav("temp.wav", self.fs, x)
        speech, _ = sf.read("temp.wav")
        return speech
    
    def get_transcription(self, file):
        speech = self.get_sample(file)
        input_features = self.processor(
            speech,
            sampling_rate=self.fs,
            return_tensors="pt"
        ).input_features.cuda()  # Batch size 1
        generated_ids = self.model.generate(inputs=input_features)
        transcription = self.processor.batch_decode(generated_ids)
        return transcription

In [7]:
base_dir = "/mntcephfs/lab_data/shoinoue/Dataset/L2-ARCTIC/"
spk = "ASI"

In [19]:
models = {
    "facebook": FacebookASR()
}

In [14]:
files = glob.glob(base_dir + f"{spk}/transcript/*")
files.sort()
groundtruth_dir = {}
for file in files:
    f = open(file, "r")
    text = f.read()
    key = os.path.basename(file)[:-4]
    groundtruth_dir[key] = normalizer(text)

In [26]:
save = False

result = {}
result["predtext"] = {}
result["wer"] = {}
audiofiles = glob.glob(base_dir + f"{spk}/wav/*.wav")
audiofiles.sort()
for size in models:
    print(size)
    model = models[size]
    result["predtext"][size] = {}
    result["wer"][size] = {}
    for file in tqdm(audiofiles):
        key = os.path.basename(file)[:-4]
        predtext = normalizer(models[size].get_transcription(file)[0])
        result["predtext"][size][key] = predtext
        wer = jiwer.wer(groundtruth_dir[key], [predtext])
        result["wer"][size][key] = wer
            
savefile = "./wer_facebook_result.npy"
if save:
    np.save(savefile, result)

facebook


100%|██████████| 1131/1131 [03:05<00:00,  6.09it/s]
